In [17]:
import pandas as pd
import os
import numpy as np
import re

In [18]:
# get all directories that store the csv files
root = '../pt'
all_dir = [x[0] for x in os.walk(root) if len(x[0].split(os.path.sep)) == 4]
all_dir

['../pt/MLDG/MLDG_tgt_T_alpha=0.75',
 '../pt/MLDG/MLDG_tgt_M_alpha=0.5',
 '../pt/MLDG/MLDG_tgt_A_alpha=0.5',
 '../pt/MLDG/MLDG_tgt_F_alpha=0.5',
 '../pt/MLDG/MLDG_tgt_F_alpha=0.1',
 '../pt/MLDG/MLDG_tgt_M_alpha=0.75',
 '../pt/MLDG/MLDG_tgt_T_alpha=0.1',
 '../pt/MLDG/MLDG_tgt_M_alpha=1',
 '../pt/MLDG/MLDG_tgt_M_alpha=0.1',
 '../pt/MLDG/MLDG_tgt_A_alpha=0.75',
 '../pt/MLDG/MLDG_tgt_A_alpha=0.1',
 '../pt/MLDG/MLDG_tgt_F_alpha=1',
 '../pt/MLDG/MLDG_tgt_T_alpha=0.5',
 '../pt/MLDG/MLDG_tgt_T_alpha=1',
 '../pt/MLDG/MLDG_tgt_F_alpha=0.75',
 '../pt/MLDG/MLDG_tgt_A_alpha=1',
 '../pt/ERM_default/ERM_tgt_A',
 '../pt/ERM_default/ERM_tgt_M',
 '../pt/ERM_default/ERM_tgt_T',
 '../pt/ERM_default/ERM_tgt_F',
 '../pt/DANN_default/DANN_tgt_F_alpha=0.5',
 '../pt/DANN_default/DANN_tgt_T_alpha=0.75',
 '../pt/DANN_default/DANN_tgt_T_alpha=1',
 '../pt/DANN_default/DANN_tgt_T',
 '../pt/DANN_default/DANN_tgt_F_alpha=1',
 '../pt/DANN_default/DANN_tgt_T_alpha=0.5',
 '../pt/DANN_default/DANN_tgt_M_alpha=0.5',
 '../

In [19]:
# parse the directory name to get algorithm, hparams
re_folder = re.compile('([A-Z]+)_tgt_([ATMF])((_[a-z]+=\d*[.,]?\d*)*)')
re_res = [re_folder.match(os.path.basename(d)) for d in all_dir]
res = [{'algo': x.group(1), 'tgt': x.group(2), 'hparams': x.group(3)} for x in re_res]
res

[{'algo': 'MLDG', 'tgt': 'T', 'hparams': '_alpha=0.75'},
 {'algo': 'MLDG', 'tgt': 'M', 'hparams': '_alpha=0.5'},
 {'algo': 'MLDG', 'tgt': 'A', 'hparams': '_alpha=0.5'},
 {'algo': 'MLDG', 'tgt': 'F', 'hparams': '_alpha=0.5'},
 {'algo': 'MLDG', 'tgt': 'F', 'hparams': '_alpha=0.1'},
 {'algo': 'MLDG', 'tgt': 'M', 'hparams': '_alpha=0.75'},
 {'algo': 'MLDG', 'tgt': 'T', 'hparams': '_alpha=0.1'},
 {'algo': 'MLDG', 'tgt': 'M', 'hparams': '_alpha=1'},
 {'algo': 'MLDG', 'tgt': 'M', 'hparams': '_alpha=0.1'},
 {'algo': 'MLDG', 'tgt': 'A', 'hparams': '_alpha=0.75'},
 {'algo': 'MLDG', 'tgt': 'A', 'hparams': '_alpha=0.1'},
 {'algo': 'MLDG', 'tgt': 'F', 'hparams': '_alpha=1'},
 {'algo': 'MLDG', 'tgt': 'T', 'hparams': '_alpha=0.5'},
 {'algo': 'MLDG', 'tgt': 'T', 'hparams': '_alpha=1'},
 {'algo': 'MLDG', 'tgt': 'F', 'hparams': '_alpha=0.75'},
 {'algo': 'MLDG', 'tgt': 'A', 'hparams': '_alpha=1'},
 {'algo': 'ERM', 'tgt': 'A', 'hparams': ''},
 {'algo': 'ERM', 'tgt': 'M', 'hparams': ''},
 {'algo': 'ERM', '

In [26]:
# read csvs and combine results
names = ['train', 'valid', 'test']
fields = ['avg_loss', 'avg_acc', 'f1']
merged = []

for folder, meta in zip(all_dir, res):
    tr, va, te = [pd.read_csv(os.path.join(folder, f'{x}_epochs.csv'))[fields] for x in names]
    
    for table, name in zip([tr, va, te], names):
        table.columns = pd.MultiIndex.from_tuples(zip([name]*len(fields), table.columns))

    all_sets = pd.concat((tr, va, te), axis=1)
    loss_min_idx = all_sets[('valid', 'avg_loss')].argmin()
    acc_max_idx = all_sets[('valid', 'avg_acc')].argmax()
    f1_max_idx = all_sets[('valid', 'f1')].argmax()
    
    row = all_sets.iloc[[loss_min_idx]].copy()
#     row = all_sets.iloc[[acc_max_idx]].copy()
#     row = all_sets.iloc[[f1_max_idx]].copy()
    for k, v in meta.items():
        row[k] = v
    
    merged.append(row)

merged = pd.concat(merged, axis=0)
col = merged.columns.tolist()
merged = merged[col[-3:]+col[:-3]]
final = merged.set_index(['tgt', 'algo', 'hparams']).sort_index(axis=0)
display(final)

train                         valid            \
                        avg_loss   avg_acc        f1  avg_loss   avg_acc   
tgt algo   hparams                                                         
A   DANN                0.235291  0.905660  0.886960  0.158477  0.939312   
           _alpha=0.5   0.277184  0.902965  0.883283  0.197824  0.937289   
           _alpha=0.75  0.113249  0.975320  0.970419  0.231434  0.934592   
           _alpha=1     0.138168  0.967318  0.961138  0.190901  0.929872   
    ERM                 0.092056  0.969845  0.964049  0.164308  0.937964   
    MLDG   _alpha=0.1   0.179806  0.940944  0.945545  0.173660  0.936952   
           _alpha=0.5   0.177336  0.943015  0.947368  0.171715  0.940324   
           _alpha=0.75  0.174513  0.944027  0.948296  0.172274  0.936952   
           _alpha=1     0.175312  0.943931  0.948193  0.177338  0.935941   
    ORACLE              0.123611  0.961458  0.961126  0.224492  0.920000   
F   DANN                0.253278  0.896322  0.877573  0.190655  0.928109   
           _alpha=0.5   0.143861  0.954839  0.946691  0.195194  0.930506   
           _alpha=0.75  0.185562  0.941962  0.931301  0.218783  0.921160   
           _alpha=1     0.124441  0.965860  0.959700  0.223500  0.926192   
    ERM                 0.252746  0.898059  0.879785  0.193136  0.927151   
    MLDG   _alpha=0.1   0.231433  0.920231  0.916440  0.198089  0.930266   
           _alpha=0.5   0.229443  0.922688  0.919115  0.194700  0.929307   
           _alpha=0.75  0.228385  0.922977  0.919336  0.194163  0.930506   
           _alpha=1     0.228468  0.922881  0.919268  0.199530  0.928349   
    ORACLE              0.135206  0.954932  0.966978  0.099387  0.964377   
M   DANN                0.090648  0.968247  0.958542  0.165047  0.943594   
           _alpha=0.5   0.264912  0.899033  0.867185  0.191225  0.929627   
           _alpha=0.75  0.139416  0.957573  0.944356  0.193012  0.934193   
           _alpha=1     0.088778  0.975363  0.967799  0.183793  0.942788   
    ERM                 0.229450  0.906418  0.876615  0.167905  0.938222   
    MLDG   _alpha=0.1   0.177917  0.942245  0.937268  0.155182  0.943057   
           _alpha=0.5   0.178931  0.942341  0.937228  0.158660  0.941982   
           _alpha=0.75  0.178338  0.942100  0.937048  0.159205  0.942519   
           _alpha=1     0.180530  0.941570  0.936315  0.156341  0.945474   
    ORACLE              0.383354  0.839399  0.896534  0.272710  0.888493   
T   DANN                0.290579  0.881523  0.900885  0.200529  0.923484   
           _alpha=0.5   0.157659  0.948255  0.956490  0.221472  0.923131   
           _alpha=0.75  0.333377  0.865391  0.887364  0.233622  0.915021   
           _alpha=1     0.121922  0.964475  0.970111  0.241089  0.921016   
    ERM                 0.121931  0.958921  0.965425  0.213299  0.923131   
    MLDG   _alpha=0.1   0.212670  0.929740  0.945111  0.207127  0.922779   
           _alpha=0.5   0.209750  0.930781  0.945872  0.197917  0.925247   
           _alpha=0.75  0.206906  0.932031  0.946854  0.196838  0.925952   
           _alpha=1     0.206224  0.932396  0.947141  0.201575  0.926305   
    ORACLE              0.202607  0.924961  0.807330  0.143700  0.942197   

                                      test                      
                              f1  avg_loss   avg_acc        f1  
tgt algo   hparams                                              
A   DANN                0.930233  0.390786  0.817625  0.837582  
           _alpha=0.5   0.928517  0.390477  0.828750  0.845512  
           _alpha=0.75  0.924455  0.394506  0.857500  0.861348  
           _alpha=1     0.920671  0.380540  0.823125  0.840203  
    ERM                 0.928181  0.389595  0.827250  0.844404  
    MLDG   _alpha=0.1   0.927039  0.328733  0.858625  0.868074  
           _alpha=0.5   0.931048  0.343452  0.857000  0.867008  
           _alpha=0.75  0.927096  0.335034  0.858250  0.867863  
           _alpha=1     0.926185  0.348673 

In [27]:
# average over target domains
final.groupby(level=['algo', 'hparams']).mean()

train                         valid            \
                    avg_loss   avg_acc        f1  avg_loss   avg_acc   
algo   hparams                                                         
DANN                0.217449  0.912938  0.905990  0.178677  0.933625   
       _alpha=0.5   0.210904  0.926273  0.913412  0.201429  0.930138   
       _alpha=0.75  0.192901  0.935062  0.933360  0.219213  0.926241   
       _alpha=1     0.118327  0.968254  0.964687  0.209821  0.929967   
ERM                 0.174046  0.933311  0.921469  0.184662  0.931617   
MLDG   _alpha=0.1   0.200457  0.933290  0.936091  0.183515  0.933263   
       _alpha=0.5   0.198865  0.934706  0.937396  0.180748  0.934215   
       _alpha=0.75  0.197036  0.935284  0.937884  0.180620  0.933982   
       _alpha=1     0.197633  0.935194  0.937729  0.183696  0.934017   
ORACLE              0.211194  0.920188  0.907992  0.185072  0.928767   

                                  test                      
                          f1  avg_loss   avg_acc        f1  
algo   hparams                                              
DANN                0.927708  0.424573  0.834829  0.826850  
       _alpha=0.5   0.924052  0.395863  0.846690  0.839408  
       _alpha=0.75  0.919232  0.429769  0.842888  0.833484  
       _alpha=1     0.923931  0.447290  0.836983  0.826521  
ERM                 0.924458  0.388027  0.841374  0.833285  
MLDG   _alpha=0.1   0.926942  0.380700  0.856533  0.848210  
       _alpha=0.5   0.927738  0.374463  0.855760  0.848604  
       _alpha=0.75  0.927424  0.372872  0.858593  0.851007  
       _alpha=1     0.927579  0.389701  0.847341  0.840388  
ORACLE              0.920266  0.172575  0.931382  0.922733

In [28]:
# only select out the best hparam setting for each algorithm
best_hparam = final.groupby(['tgt', 'algo']).apply(lambda df: df.reset_index(level=2).iloc[df[('valid', 'avg_loss')].argmin()])
best_hparam = best_hparam.set_index('hparams', append=True)
display(best_hparam)

train                         valid            \
                        avg_loss   avg_acc        f1  avg_loss   avg_acc   
tgt algo   hparams                                                         
A   DANN                0.235291  0.905660  0.886960  0.158477  0.939312   
    ERM                 0.092056  0.969845  0.964049  0.164308  0.937964   
    MLDG   _alpha=0.5   0.177336  0.943015  0.947368  0.171715  0.940324   
    ORACLE              0.123611  0.961458  0.961126  0.224492  0.920000   
F   DANN                0.253278  0.896322  0.877573  0.190655  0.928109   
    ERM                 0.252746  0.898059  0.879785  0.193136  0.927151   
    MLDG   _alpha=0.75  0.228385  0.922977  0.919336  0.194163  0.930506   
    ORACLE              0.135206  0.954932  0.966978  0.099387  0.964377   
M   DANN                0.090648  0.968247  0.958542  0.165047  0.943594   
    ERM                 0.229450  0.906418  0.876615  0.167905  0.938222   
    MLDG   _alpha=0.1   0.177917  0.942245  0.937268  0.155182  0.943057   
    ORACLE              0.383354  0.839399  0.896534  0.272710  0.888493   
T   DANN                0.290579  0.881523  0.900885  0.200529  0.923484   
    ERM                 0.121931  0.958921  0.965425  0.213299  0.923131   
    MLDG   _alpha=0.75  0.206906  0.932031  0.946854  0.196838  0.925952   
    ORACLE              0.202607  0.924961  0.807330  0.143700  0.942197   

                                      test                      
                              f1  avg_loss   avg_acc        f1  
tgt algo   hparams                                              
A   DANN                0.930233  0.390786  0.817625  0.837582  
    ERM                 0.928181  0.389595  0.827250  0.844404  
    MLDG   _alpha=0.5   0.931048  0.343452  0.857000  0.867008  
    ORACLE              0.922424  0.201909  0.926250  0.925693  
F   DANN                0.916944  0.296331  0.897306  0.927286  
    ERM                 0.914989  0.268290  0.905948  0.932898  
    MLDG   _alpha=0.75  0.918448  0.318031  0.884087  0.918455  
    ORACLE              0.973684  0.111441  0.951777  0.967071  
M   DANN                0.927536  0.571103  0.803745  0.852931  
    ERM                 0.918033  0.532312  0.782650  0.833060  
    MLDG   _alpha=0.1   0.925978  0.539407  0.813226  0.863099  
    ORACLE              0.922697  0.241529  0.901775  0.933226  
T   DANN                0.936120  0.440070  0.820640  0.689600  
    ERM                 0.936628  0.361913  0.849648  0.722779  
    MLDG   _alpha=0.75  0.938561  0.317892  0.872645  0.749545  
    ORACLE              0.862259  0.135423  0.945727  0.864943

In [29]:
best_hparam.groupby(level=['algo']).mean()

train                         valid                          test  \
        avg_loss   avg_acc        f1  avg_loss   avg_acc        f1  avg_loss   
algo                                                                           
DANN    0.217449  0.912938  0.905990  0.178677  0.933625  0.927708  0.424573   
ERM     0.174046  0.933311  0.921469  0.184662  0.931617  0.924458  0.388027   
MLDG    0.197636  0.935067  0.937707  0.179474  0.934960  0.928508  0.379695   
ORACLE  0.211194  0.920188  0.907992  0.185072  0.928767  0.920266  0.172575   

                            
         avg_acc        f1  
algo                        
DANN    0.834829  0.826850  
ERM     0.841374  0.833285  
MLDG    0.856740  0.849527  
ORACLE  0.931382  0.922733